In [ ]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

# ================= 配置 (5090 顶配) =================
MAX_SEQ_LENGTH = 2048
DTYPE = None # 自动检测 (BF16)
LOAD_IN_4BIT = True

DATA_FILE = "gold_llm_train.jsonl"
OUTPUT_DIR = "llama3_gold_quant_checkpoint"

print(f"🔥 唤醒 RTX 5090... CUDA: {torch.cuda.is_available()}")

# 1. 加载模型
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", 
    max_seq_length = MAX_SEQ_LENGTH,
    dtype = DTYPE,
    load_in_4bit = LOAD_IN_4BIT,
)

# 2. 配置 LoRA
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # 高 Rank 以捕捉细微的宏观逻辑
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

# 3. 数据加载
dataset = load_dataset("json", data_files=DATA_FILE, split="train")

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + tokenizer.eos_token
        texts.append(text)
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

# 4. 训练参数 (Batch Size 拉满)
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = MAX_SEQ_LENGTH,
    dataset_num_proc = 4,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 16, # 5090 显存大，直接 16
        gradient_accumulation_steps = 1,
        warmup_steps = 10,
        max_steps = 250, # 这里的步数取决于你的数据量，200步大概几分钟跑完用于演示
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = OUTPUT_DIR,
    ),
)

print("🚀 开始训练... (AI 宏观分析师正在学习历史规律)")
trainer.train()

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)
print(f"✅ 模型已保存至: {OUTPUT_DIR}")